In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import time
import collections
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor, tensor
import pandas as pd
import openml

#from aeon.regression.sklearn import RotationForestRegressor
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

from preprocessing.stream_transforms import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from utils.utils import print_name, print_shape
from models import ResNet, NeuralEulerODE, RidgeClassifierCVModule, E2EResNet, LogisticRegressionSGD, RandFeatBoost

np.set_printoptions(precision=3, threshold=5) # Print options
device = "cuda" # torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# MNIST

In [2]:
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Download and load the training data
mnist_path = "/home/nikita/hdd/MNIST"
trainset = datasets.MNIST(mnist_path, download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=len(trainset), shuffle=False)

# Download and load the test data
testset = datasets.MNIST(mnist_path, download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=len(testset), shuffle=False)

# Flatten the data
X_train, y_train_cat = next(iter(trainloader))
X_train = X_train.view(len(trainset), -1).to(device)
X_test, y_test_cat = next(iter(testloader))
X_test = X_test.view(len(testset), -1).to(device)

# Convert train and test labels to one-hot encoding
y_train = nn.functional.one_hot(y_train_cat, num_classes=10).float().to(device)
y_test = nn.functional.one_hot(y_test_cat, num_classes=10).float().to(device)
y_train_cat = y_train_cat.to(device)
y_test_cat = y_test_cat.to(device)

# Normalize by mean and std
X_train, X_test = normalize_mean_std_traindata(X_train, X_test)
print(f"Train data shape: {X_train.shape}")
print(f"Train labels shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Test labels shape: {y_test.shape}")

Train data shape: torch.Size([60000, 784])
Train labels shape: torch.Size([60000, 10])
Test data shape: torch.Size([10000, 784])
Test labels shape: torch.Size([10000, 10])


# Logistic Regression

In [3]:
from models import LogisticRegression

N = 100
D = 50
C = 10
gen = torch.Generator().manual_seed(42)
X = torch.randn(N, D, generator=gen)
y = torch.randint(0, C, size=(N,), generator=gen)
model = LogisticRegression(
        gen,
        in_dim = D,
        out_dim = C,
        l2_reg = 1.0,
        max_iter = 100,
    )
_, _ = model.fit(X, y)

# Gradient Random Feature Boosting

In [4]:
from models import GradientRandomFeatureBoostingClassification

N = 100
D = 50
C = 10
bottleneck_dim = 70
gen = torch.Generator().manual_seed(42)
X = torch.randn(N, D, generator=gen)
y = torch.randint(0, C, size=(N,), generator=gen)
y = nn.functional.one_hot(y, num_classes=C).float()
model = GradientRandomFeatureBoostingClassification(
        gen,
        hidden_dim = D,
        bottleneck_dim = bottleneck_dim,
        out_dim = C,
        n_layers = 2,
        upscale = "dense",
        feature_type = "dense",
    )
_, _ = model.fit(X, y)

alpha 10.0
linesearch loss tensor(2.0531, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0531, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0531, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0531, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0531, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0531, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0531, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0530, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0530, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0530, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0530, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0530, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0530, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0530, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0530, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0530, grad_fn=<NllLossBackward0>)
linesearch loss tensor(2.0530, grad_fn=<NllLo

# experiments

In [5]:
def run_allmodels_1dataset(
        generator: torch.Generator,
        X_train: Tensor,
        y_train: Tensor,
        X_test: Tensor,
        y_test: Tensor,
        ):
    
    D = X_train.shape[1]
    hidden_size = 512
    bottleneck_dim = 1*hidden_size
    num_epochs = 50
    batch_size = 128

    # (name, model, kwargs). kwargs separate to save memory
    model_list = [
        # ["T=10 RandFeatureBoost", RandFeatBoost,
        #         {"generator": generator,
        #          "in_dim": D,
        #          "hidden_size": hidden_size,
        #          "out_dim": 10,
        #          "n_blocks": 9,
        #          "activation": nn.Tanh(),
        #          "loss_fn": nn.CrossEntropyLoss(),
        #          "adam_lr": 1e-2,
        #          "boost_lr": 1.0,
        #          "epochs": num_epochs,
        #          "batch_size": batch_size,
        #          "upscale_type": "SWIM",  # "dense", "identity"
        #          }],

        # ["Tabular Ridge", RidgeClassifierCVModule, {}],

        # ["Logistic SGD", LogisticRegressionSGD, 
        #         {"generator": generator,
        #          "num_epochs": num_epochs,
        #          "batch_size": batch_size,
        #          }],

        # ["Logistic L-BFSG", LogisticRegression, 
        #         {"generator": generator,
        #          "in_dim": D,
        #          "out_dim": 10,
        #          }],

        # ["T=1 Dense", ResNet,
        #         {"generator": generator,
        #          "in_dim": D,
        #          "hidden_size": hidden_size,
        #          "bottleneck_dim": None,
        #          "n_blocks": 0,
        #          "upsample_layer": "dense",
        #          "output_layer": "logistic regression",
        #          }],

        # ["T=1 SWIM Grad", ResNet,
        #         {"generator": generator,
        #         "in_dim": D,
        #         "hidden_size": hidden_size,
        #         "bottleneck_dim": None,
        #         "n_blocks": 0,
        #         "upsample_layer": "SWIM",
        #         "output_layer": "logistic regression",
        #         }],
        
        # ["T=1 SWIM Unif", ResNet,
        #         {"generator": generator,
        #         "in_dim": D,
        #         "hidden_size": hidden_size,
        #         "bottleneck_dim": None,
        #         "n_blocks": 0,
        #         "upsample_layer": "SWIM",
        #         "sampling_method": "uniform",
        #         "output_layer": "logistic regression",
        #         }],

    ]
    for lr in [1.0]:
        for l2_reg in [1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]:
            model_list += [
                [f"Logistic L-BFSG, l2={l2_reg} lr={lr}", 
                    LogisticRegression, 
                    {"generator": generator,
                    "in_dim": D,
                    "out_dim": 10,
                    "l2_reg": l2_reg,
                    "lr": lr,
                    }],
            ]
    
    for n_blocks in range(0, 20, 5):
        model_list += [
            [f"T={n_blocks+1} Gradient GRFBoost", 
             GradientRandomFeatureBoostingClassification,
                {"generator": generator,
                "hidden_dim": hidden_size,
                "bottleneck_dim": bottleneck_dim,
                "out_dim": 10,
                "n_layers": n_blocks,
                "activation": nn.Tanh(),
                "l2_reg": 0.00001,
                "feature_type": "SWIM",
                "boost_lr": 0.5,
                "upscale": "SWIM",
                },
                ],
        ]

    # for n_blocks in [4]:
    #     model_list += [
    #     [f"T={n_blocks+1} End2End", E2EResNet,
    #             {"generator": generator,
    #             "in_dim": D,
    #             "hidden_size": hidden_size,
    #             "bottleneck_dim": bottleneck_dim,
    #             "out_dim": 10,
    #             "n_blocks": n_blocks,
    #             "activation": nn.Tanh(),
    #             "loss": nn.CrossEntropyLoss(),
    #             "lr": 1e-2,
    #             "epochs": num_epochs,
    #             "batch_size": batch_size}
    #             ],

    #     [f"T={n_blocks+1} ResSWIM Grad-dense", ResNet,
    #             {"generator": generator,
    #             "in_dim": D,
    #             "hidden_size": hidden_size,
    #             "bottleneck_dim": bottleneck_dim,
    #             "n_blocks": n_blocks,
    #             "upsample_layer": "SWIM",
    #             "res_layer1": "SWIM",
    #             "res_layer2": "dense",
    #             "output_layer": "logistic regression",
    #             }
    #             ],

    #     [f"T={n_blocks+1} ResSWIM Grad-id", ResNet,
    #             {"generator": generator,
    #             "in_dim": D,
    #             "hidden_size": hidden_size,
    #             "bottleneck_dim": hidden_size,
    #             "n_blocks": n_blocks,
    #             "upsample_layer": "SWIM",
    #             "res_layer1": "SWIM",
    #             "res_layer2": "identity",
    #             "output_layer": "logistic regression",
    #             }
    #             ],

    #     [f"T={n_blocks+1} ResDense", ResNet,
    #             {"generator": generator,
    #             "in_dim": D,
    #             "hidden_size": hidden_size,
    #             "bottleneck_dim": hidden_size,
    #             "n_blocks": n_blocks,
    #             "upsample_layer": "dense",
    #             "res_layer1": "dense",
    #             "res_layer2": "identity",
    #             "output_layer": "logistic regression",
    #             }
    #             ],
    # ]
    
    results = []
    model_names = []
    for name, model, model_args in model_list:
        print(name)
        t0 = time.perf_counter()
        model = model(**model_args).to(X_train.device)
        pred_train, _ = model.fit(X_train, y_train)
        t1 = time.perf_counter()
        pred_test = model(X_test)
        t2 = time.perf_counter()
        
        #convert to class predictions:
        if len(pred_train.shape) == 2:
            pred_train = torch.argmax(pred_train, dim=1)
            pred_test = torch.argmax(pred_test, dim=1)
        acc_train = (pred_train == y_train_cat).float().mean().item()
        acc_test = (pred_test == y_test_cat).float().mean().item()

        result = np.array( [acc_train, acc_test, t1-t0, t2-t1] )
        results.append( result )
        model_names.append( name )

    return model_names, results



def run_all_experiments(
        name_save: str = "PLACEHOLDER",
        ):
    # Fetch and process each dataset
    experiments = {}
    generator = torch.Generator(device=device).manual_seed(999)
    results = run_allmodels_1dataset(
        generator, X_train, y_train, X_test, y_test, 
        )
    experiments["MNIST"] = results

    # Save results
    # Assuming experiments is a dict where keys are dataset names and values are tuples (model_names, results)
    attributes = ["acc_train", "acc_test", "t_fit", "t_feat"]
    data_list = []
    # Process the data
    for dataset_name, (model_names, results) in experiments.items():
        dataset_data = {}
        for attr_idx, attribute in enumerate(attributes):
            for model_idx, model_name in enumerate(model_names):
                dataset_data[(attribute, model_name)] = results[model_idx][attr_idx]
        data_list.append(pd.DataFrame(dataset_data, index=[dataset_name]))

    # Combine all datasets into a single DataFrame
    df = pd.concat(data_list)
    df = df.sort_index(axis=1)
    print(df)
    df.to_pickle(f"MNIST_{name_save}.pkl")
    return df

In [6]:
_ = run_all_experiments(name_save="TESTING")

Logistic L-BFSG, l2=1 lr=1.0
Logistic L-BFSG, l2=0.1 lr=1.0
Logistic L-BFSG, l2=0.01 lr=1.0
Logistic L-BFSG, l2=0.001 lr=1.0
Logistic L-BFSG, l2=0.0001 lr=1.0
Logistic L-BFSG, l2=1e-05 lr=1.0
Logistic L-BFSG, l2=1e-06 lr=1.0
T=1 Gradient GRFBoost
T=6 Gradient GRFBoost
alpha 0.009999999776482582
linesearch loss tensor(0.2755, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(5.7010, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.9132, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(12.8609, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.3152, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.1762, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.1516, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.1502, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.1501, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss ten

In [7]:
df = pd.read_pickle("MNIST_TESTING.pkl")
df["acc_test"].mean().sort_values(ascending=False)

T=16 Gradient GRFBoost               0.9690
T=11 Gradient GRFBoost               0.9684
T=6 Gradient GRFBoost                0.9649
Logistic L-BFSG, l2=1e-05 lr=1.0     0.9243
Logistic L-BFSG, l2=0.0001 lr=1.0    0.9238
Logistic L-BFSG, l2=0.001 lr=1.0     0.9231
T=1 Gradient GRFBoost                0.9226
Logistic L-BFSG, l2=1e-06 lr=1.0     0.9220
Logistic L-BFSG, l2=0.01 lr=1.0      0.9174
Logistic L-BFSG, l2=0.1 lr=1.0       0.8991
Logistic L-BFSG, l2=1 lr=1.0         0.8480
dtype: float64

In [8]:
# T=5 End2End        0.9717
# T=1 Dense          0.9215
# T=1 SWIM Unif      0.9207
# T=1 SWIM Grad      0.9204
# Logistic SGD       0.8990
# Tabular Ridge      0.8606
# Logistic L-BFSG    0.8480
# dtype: float64

In [9]:
df["acc_train"].mean().sort_values(ascending=False)

T=16 Gradient GRFBoost               0.978417
T=11 Gradient GRFBoost               0.976083
T=6 Gradient GRFBoost                0.973017
Logistic L-BFSG, l2=1e-05 lr=1.0     0.931450
Logistic L-BFSG, l2=1e-06 lr=1.0     0.931200
Logistic L-BFSG, l2=0.0001 lr=1.0    0.931000
Logistic L-BFSG, l2=0.001 lr=1.0     0.930167
T=1 Gradient GRFBoost                0.922150
Logistic L-BFSG, l2=0.01 lr=1.0      0.918317
Logistic L-BFSG, l2=0.1 lr=1.0       0.894617
Logistic L-BFSG, l2=1 lr=1.0         0.838183
dtype: float64

# Experiment layer by layer with Gradient Boosting

In [10]:
# Time to experiment with RandFeatureBoost


generator = torch.Generator(device=device).manual_seed(999)
D = X_train.shape[1]
hidden_size = 512
n_blocks = 20
boost_lr=1.0

model = GradientRandomFeatureBoostingClassification(
    generator=generator,
    hidden_dim=hidden_size,
    out_dim=10,
    n_layers=n_blocks,
    l2_reg=0.0001,
    boost_lr=boost_lr,
    feature_type="SWIM",
    upscale="SWIM",
    ).to(device)

pred_train, _ = model.fit(X_train, y_train)
pred_test = model(X_test)
pred_train = torch.argmax(pred_train, dim=1)
pred_test = torch.argmax(pred_test, dim=1)
acc_train = (pred_train == y_train_cat).float().mean().item()
acc_test = (pred_test == y_test_cat).float().mean().item()

alpha 0.009999999776482582
linesearch loss tensor(0.3351, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2944, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2499, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2431, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2429, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2429, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2429, device='cuda:0', grad_fn=<NllLossBackward0>)
Linesearch 0.33945271372795105
Gradient hat norm tensor(198.0060, device='cuda:0')
alpha 0.009999999776482582
linesearch loss tensor(0.2977, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2372, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2181, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.2154, device='cuda:0', grad_fn=<NllLossBackward0>)
linesearch loss tensor(0.21

In [11]:
print(acc_train)
print(acc_test)

0.9626833200454712
0.9571999907493591


In [12]:
def print_all_accuracies(X_train, X_test):
    with torch.no_grad():
        X_train = model.upscale(X_train)
        X_test  = model.upscale(X_test)

        for t, (layer, (Delta, Delta_b), cls) in enumerate(zip(model.layers, model.deltas, model.classifiers[1:])):
            X_train += model.boost_lr * (layer(X_train)@Delta + Delta_b )
            X_test +=  model.boost_lr * (layer(X_test)@Delta + Delta_b )

            #delta norm

            pred_train = cls(X_train)
            pred_test = cls(X_test)
            pred_train = torch.argmax(pred_train, dim=1)
            pred_test = torch.argmax(pred_test, dim=1)
            acc_train = (pred_train == y_train_cat).float().mean().item()
            acc_test = (pred_test == y_test_cat).float().mean().item()

            print(f"Block {t}: Train acc: {acc_train}, Test acc: {acc_test}")
            # print("delta norm", torch.linalg.norm(Delta).item())
            # print("X_train norm", torch.linalg.norm(X_train).item() / X_train.size(0))
            # print("X_test norm", torch.linalg.norm(X_test).item() / X_test.size(0))
            

print_all_accuracies(X_train, X_test)

Block 0: Train acc: 0.9140666723251343, Test acc: 0.9178999662399292
Block 1: Train acc: 0.9391999840736389, Test acc: 0.9405999779701233
Block 2: Train acc: 0.949066698551178, Test acc: 0.9469999670982361
Block 3: Train acc: 0.9535666704177856, Test acc: 0.9508000016212463
Block 4: Train acc: 0.95660001039505, Test acc: 0.9531999826431274
Block 5: Train acc: 0.9593999981880188, Test acc: 0.9551999568939209
Block 6: Train acc: 0.9609000086784363, Test acc: 0.9566999673843384
Block 7: Train acc: 0.9617166519165039, Test acc: 0.9573999643325806
Block 8: Train acc: 0.9622666835784912, Test acc: 0.9566999673843384
Block 9: Train acc: 0.9624500274658203, Test acc: 0.9565999507904053
Block 10: Train acc: 0.9620500206947327, Test acc: 0.9563999772071838
Block 11: Train acc: 0.9622333645820618, Test acc: 0.9565999507904053
Block 12: Train acc: 0.9622666835784912, Test acc: 0.9563999772071838
Block 13: Train acc: 0.9622833728790283, Test acc: 0.9565999507904053
Block 14: Train acc: 0.9623667001

In [13]:
# idea: normalize the gradient before fitting the next layer. This is to find the optimal direction. Then do line search

# rand feat boost

In [14]:
# Time to experiment with RandFeatureBoost

generator = torch.Generator(device=device).manual_seed(999)
D = X_train.shape[1]
hidden_size = 128
num_epochs = 30
batch_size = 512
n_blocks = 5
adam_lr=1e-2
boost_lr=0.9

model = RandFeatBoost(
    generator=generator,
    in_dim=D,
    hidden_size=hidden_size,
    out_dim=10,
    n_blocks=n_blocks,
    activation=nn.Tanh(),
    loss_fn=nn.CrossEntropyLoss(),
    adam_lr=adam_lr,
    boost_lr=boost_lr,
    epochs=num_epochs,
    batch_size=batch_size,
    upscale_type="SWIM",
    ).to(device)

pred_train, _ = model.fit(X_train, y_train)
pred_test = model(X_test)
pred_train = torch.argmax(pred_train, dim=1)
pred_test = torch.argmax(pred_test, dim=1)
acc_train = (pred_train == y_train_cat).float().mean().item()
acc_test = (pred_test == y_test_cat).float().mean().item()

 17%|█▋        | 5/30 [00:03<00:19,  1.27it/s]


KeyboardInterrupt: 

In [ ]:
print(acc_train)
print(acc_test)

In [ ]:
model.deltas

In [ ]:
def print_all_accuracies(X_train, X_test):
    X_train = model.upscale(X_train)
    X_test  = model.upscale(X_test)

    for t, (layer, DELTA, classifier) in enumerate(zip(model.layers, model.deltas, model.classifiers)):
        X_train = X_train + model.boost_lr * DELTA * (layer(X_train) - X_train)
        X_test = X_test + model.boost_lr * DELTA * (layer(X_test) - X_test)
        
        pred_train = classifier(X_train)
        pred_test = classifier(X_test)
        pred_train = torch.argmax(pred_train, dim=1)
        pred_test = torch.argmax(pred_test, dim=1)
        acc_train = (pred_train == y_train_cat).float().mean().item()
        acc_test = (pred_test == y_test_cat).float().mean().item()

        print(f"Block {t}: Train acc: {acc_train}, Test acc: {acc_test}")


print_all_accuracies(X_train, X_test)

In [ ]:
# Test SWIM-ID vs DENSE-ID vs SWIM-DENSE 
# implement 'finding gradient direction' gradient boosting

# Test whether this is actually better than non-boost with same hidden size !!!!!!!!!!!!!!!!!!!!!

In [ ]:
# experiment with DENSE

generator = torch.Generator(device=device).manual_seed(999)
D = X_train.shape[1]
hidden_size = 800
num_epochs = 50
batch_size = 128
n_blocks = 10
adam_lr=1e-2
boost_lr=0.9

model = RandFeatBoost(
    generator=generator,
    in_dim=D,
    hidden_size=hidden_size,
    out_dim=10,
    n_blocks=n_blocks,
    activation=nn.Tanh(),
    loss_fn=nn.CrossEntropyLoss(),
    adam_lr=adam_lr,
    boost_lr=boost_lr,
    epochs=num_epochs,
    batch_size=batch_size,
    upscale_type="dense",
    second_in_resblock="identity",
    ).to(device)

pred_train, _ = model.fit(X_train, y_train)
pred_test = model(X_test)
pred_train = torch.argmax(pred_train, dim=1)
pred_test = torch.argmax(pred_test, dim=1)
acc_train = (pred_train == y_train_cat).float().mean().item()
acc_test = (pred_test == y_test_cat).float().mean().item()

In [ ]:
def print_all_accuracies(X_train, X_test):
    X_train = model.upscale(X_train)
    X_test  = model.upscale(X_test)

    for t, (layer, DELTA, classifier) in enumerate(zip(model.layers, model.deltas, model.classifiers)):
        X_train = X_train + model.boost_lr * DELTA * (layer(X_train) - X_train)
        X_test = X_test + model.boost_lr * DELTA * (layer(X_test) - X_test)
        
        pred_train = classifier(X_train)
        pred_test = classifier(X_test)
        pred_train = torch.argmax(pred_train, dim=1)
        pred_test = torch.argmax(pred_test, dim=1)
        acc_train = (pred_train == y_train_cat).float().mean().item()
        acc_test = (pred_test == y_test_cat).float().mean().item()

        print(f"Block {t}: Train acc: {acc_train}, Test acc: {acc_test}")

print_all_accuracies(X_train, X_test)

In [ ]:
# NEXT TIME: TODO TODO TODO TODO TODO

# do gradient boosting for BINARY CLASSIFICATION
 
# do f(x_t, x_0) and not just f(x_t)

# xgboost model

# optuna (with xgboost to start with?)